In [2]:
!pip install autogluon

     ---------------------------------------- 0.0/61.0 kB ? eta -:--:--
     ---------------------------------------- 61.0/61.0 kB 3.4 MB/s eta 0:00:00
     ---------------------------------------- 0.0/60.4 kB ? eta -:--:--
     ---------------------------------------- 60.4/60.4 kB ? eta 0:00:00
     ---------------------------------------- 0.0/57.6 kB ? eta -:--:--
     ---------------------------------------- 57.6/57.6 kB 3.0 MB/s eta 0:00:00
     ---------------------------------------- 0.0/57.2 kB ? eta -:--:--
     ---------------------------------------- 57.2/57.2 kB 3.1 MB/s eta 0:00:00
     ---------------------------------------- 0.0/130.7 kB ? eta -:--:--
     -------------------------------------- 130.7/130.7 kB 8.0 MB/s eta 0:00:00
  Using cached jsonschema-4.21.1-py3-none-any.whl.metadata (7.8 kB)
     ---------------------------------------- 0.0/43.6 kB ? eta -:--:--
     ---------------------------------------- 43.6/43.6 kB ? eta 0:00:00
  Preparing metadata (setup.py): 

In [28]:
from autogluon.timeseries import TimeSeriesPredictor,TimeSeriesDataFrame
import pandas as pd
import numpy as np

In [29]:
train_df = pd.read_csv("C:/Users/hiro7/OneDrive/바탕 화면/공부,실습/AI프레임워크 과제/python_folder/input/store-sales-time-series-forecasting/train.csv")
test_df = pd.read_csv("C:/Users/hiro7/OneDrive/바탕 화면/공부,실습/AI프레임워크 과제/python_folder/input/store-sales-time-series-forecasting/test.csv")
holiday_df = pd.read_csv("C:/Users/hiro7/OneDrive/바탕 화면/공부,실습/AI프레임워크 과제/python_folder/input/store-sales-time-series-forecasting/holidays_events.csv")
oil_df =pd.read_csv("C:/Users/hiro7/OneDrive/바탕 화면/공부,실습/AI프레임워크 과제/python_folder/input/store-sales-time-series-forecasting/oil.csv")
stores_df = pd.read_csv("C:/Users/hiro7/OneDrive/바탕 화면/공부,실습/AI프레임워크 과제/python_folder/input/store-sales-time-series-forecasting/stores.csv")
submission_df =pd.read_csv("C:/Users/hiro7/OneDrive/바탕 화면/공부,실습/AI프레임워크 과제/python_folder/input/store-sales-time-series-forecasting/sample_submission.csv")

In [30]:
train_df.head()

,id,date,store_nbr,family,sales,onpromotion
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0
1,1,2013-01-01,1,BABY CARE,0.0,0
2,2,2013-01-01,1,BEAUTY,0.0,0
3,3,2013-01-01,1,BEVERAGES,0.0,0
4,4,2013-01-01,1,BOOKS,0.0,0


In [31]:
# Convert 'date' column to datetime format
train_df['date'] = pd.to_datetime(train_df['date'])
test_df['date'] = pd.to_datetime(test_df['date'])

In [32]:
# Sorting  the data 
train_df = train_df.sort_values(by=['store_nbr', 'family', 'date'])
test_df = test_df.sort_values(by=['store_nbr'
, 'family', 'date'])

In [33]:
# Dropping the id column 
train_df = train_df.drop(columns=['id'])
test_df = test_df.drop(columns=['id'])

In [34]:
# This is the sorted data 
train_df.head()

,date,store_nbr,family,sales,onpromotion
0,2013-01-01,1,AUTOMOTIVE,0.0,0
1782,2013-01-02,1,AUTOMOTIVE,2.0,0
3564,2013-01-03,1,AUTOMOTIVE,3.0,0
5346,2013-01-04,1,AUTOMOTIVE,3.0,0
7128,2013-01-05,1,AUTOMOTIVE,5.0,0


In [8]:
# Renaming the columns as per the AUtogluon requirements
train_df = train_df.rename(columns={'sales': 'target', 'date': 'timestamp','family':'item_id'})

In [9]:
test_df = test_df.rename(columns = {'date': 'timestamp','family':'item_id'})

In [10]:
train_df_single_var = train_df.drop(columns=['store_nbr','onpromotion'])

In [11]:
train_df_single_var.head()

,timestamp,item_id,target
0,2013-01-01,AUTOMOTIVE,0.0
1782,2013-01-02,AUTOMOTIVE,2.0
3564,2013-01-03,AUTOMOTIVE,3.0
5346,2013-01-04,AUTOMOTIVE,3.0
7128,2013-01-05,AUTOMOTIVE,5.0


In [12]:
train_data = TimeSeriesDataFrame.from_data_frame(
    train_df_single_var,
    id_column="item_id",
    timestamp_column="timestamp"
)
train_data.head()

target
item_id    timestamp         
AUTOMOTIVE 2013-01-01     0.0
           2013-01-02     2.0
           2013-01-03     3.0
           2013-01-04     3.0
           2013-01-05     5.0

In [13]:
train_data.describe()

,target
count,3.000888e+06
mean,3.577757e+02
std,1.101998e+03
min,0.000000e+00
25%,0.000000e+00
50%,1.100000e+01
75%,1.958473e+02
max,1.247170e+05


In [14]:
# Initialize the TimeSeriesPredictor
predictor = TimeSeriesPredictor(
    target="target",
    prediction_length=16,
    freq = 'D',
    eval_metric='RMSLE',
)

In [15]:
predictor.fit(train_data)

Beginning AutoGluon training...
AutoGluon will save models to 'AutogluonModels\ag-20240606_121411'
=================== System Info ===================
AutoGluon Version:  1.1.0
Python Version:     3.11.5
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.22631
CPU Count:          12
GPU Count:          0
Memory Avail:       6.49 GB / 15.85 GB (41.0%)
Disk Space Avail:   271.19 GB / 464.54 GB (58.4%)

Fitting with arguments:
{'enable_ensemble': True,
 'eval_metric': RMSLE,
 'freq': 'D',
 'hyperparameters': 'default',
 'known_covariates_names': [],
 'num_val_windows': 1,
 'prediction_length': 16,
 'quantile_levels': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
 'random_seed': 123,
 'refit_every_n_windows': 1,
 'refit_full': False,
 'skip_model_selection': False,
 'target': 'target',
 'verbosity': 2}

train_data with frequency 'None' has been resampled to frequency 'D'.
Provided train_data has 55704 rows (NaN fraction=0.2%), 33 time series. Median time series

In [16]:
predictor.leaderboard(train_data, silent=True)

data with frequency 'None' has been resampled to frequency 'D'.
Additional data provided, testing on additional data. Resulting leaderboard will be sorted according to test score (`score_test`).


,model,score_test,score_val,pred_time_test,pred_time_val,fit_time_marginal,fit_order
0,WeightedEnsemble,-0.184466,-0.184466,83.935260,85.800056,1.607457,12
1,AutoARIMA,-0.195445,-0.195445,49.978337,46.779015,0.066039,8
2,AutoETS,-0.202739,-0.202739,50.984877,23.618367,0.069000,7
3,DynamicOptimizedTheta,-0.205706,-0.205706,33.011748,37.054020,0.064997,6
4,RecursiveTabular,-0.211128,-0.211128,0.933618,1.967021,109.297318,2
5,DirectTabular,-0.320423,-0.320423,0.265379,0.285002,1.332516,3
6,CrostonSBA,-0.374136,-0.374136,5.272071,14.928659,0.064003,4
7,SeasonalNaive,-0.401977,-0.401977,0.104757,5.327135,0.070998,1
8,DeepAR,-0.412532,-0.420839,0.581453,0.540602,161.487685,10
9,PatchTST,-0.426361,-0.426361,0.110296,0.074810,41.505952,11


In [17]:
# Make predictions on the train data with the future of 16 days 
predictions = predictor.predict(train_data)

data with frequency 'None' has been resampled to frequency 'D'.
Model not specified in predict, will default to the model with the best validation score: WeightedEnsemble


In [18]:
predictions.head()

mean       0.1       0.2       0.3       0.4  \
item_id    timestamp                                                      
AUTOMOTIVE 2017-08-16  5.917114  4.580706  5.012063  5.325822  5.656525   
           2017-08-17  5.801718  4.276870  4.839953  5.172626  5.460407   
           2017-08-18  6.672486  4.958948  5.580505  5.995181  6.351051   
           2017-08-19  9.007484  7.373209  7.879071  8.264002  8.608279   
           2017-08-20  9.560295  7.948094  8.516014  8.890560  9.225638   

                            0.5       0.6        0.7        0.8        0.9  
item_id    timestamp                                                        
AUTOMOTIVE 2017-08-16  5.942814  6.239169   6.529812   6.862280   7.378055  
           2017-08-17  5.725946  6.044489   6.311015   6.658095   7.191466  
           2017-08-18  6.616950  6.937687   7.244122   7.634529   8.175970  
           2017-08-19  8.919519  9.231287   9.556074   9.947759  10.502207  
           2017-08-20  9.534644  9.843172  10.185974  10.610451  11.242520

In [19]:
test_df_single_var = test_df.drop(columns=['store_nbr','onpromotion'])

In [20]:
test_df_single_var.head()

,timestamp,item_id
0,2017-08-16,AUTOMOTIVE
1782,2017-08-17,AUTOMOTIVE
3564,2017-08-18,AUTOMOTIVE
5346,2017-08-19,AUTOMOTIVE
7128,2017-08-20,AUTOMOTIVE


In [21]:
# Joining the table 
mean_in_order = test_df_single_var.merge(predictions, on=['item_id', 'timestamp'], how='left')['mean']
mean_in_order

0         5.917114
1         5.801718
2         6.672486
3         9.007484
4         9.560295
           ...    
28507    24.592598
28508    18.174098
28509    16.896252
28510    17.894673
28511    14.673952
Name: mean, Length: 28512, dtype: float64

In [22]:
submission_df

,id,sales
0,3000888,0.0
1,3000889,0.0
2,3000890,0.0
3,3000891,0.0
4,3000892,0.0
...,...,...
28507,3029395,0.0
28508,3029396,0.0
28509,3029397,0.0
28510,3029398,0.0


In [23]:
submission_df['sales']= mean_in_order

In [25]:
!rm -rf ./working/*

'rm'��(��) ���� �Ǵ� �ܺ� ����, ������ �� �ִ� ���α׷�, �Ǵ�
��ġ ������ �ƴմϴ�.


In [27]:
submission_df.to_csv('./working/submission.csv', index=False)